In [3]:
import torch
from torch import nn, optim
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from torchinfo import summary
import pandas as pd
from torchvision import datasets
import os
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import confusion_matrix, accuracy_score
from torchvision import models, transforms, utils
import os
import torchvision
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [ ]:
df = pd.read_csv("mnist_train.csv")

In [ ]:
x = df[:,:-1]
y = df[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
class class_Dataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        
        # Reshape the feature into the original image shape (1, 28, 28) for MNIST
        feature = feature.view(1, 48, 48)
        
        if self.transform:
            feature = self.transform(feature)
        
        return feature, label

In [ ]:
# Define transformations for the images (e.g., resizing, normalization)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

In [ ]:
# Create an instance of the custom dataset
train_dataset = class_Dataset(X_train, y_train,transform=transform)
test_dataset = class_Dataset(X_test, y_test,transform=transform)
# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [4]:
BATCH_SIZE = 128

In [ ]:
print(f"Length of training dataloader: {len(train_loader)} batches of {BATCH_SIZE}")
print(f"Length of testing dataloader: {len(test_loader)} batches of {BATCH_SIZE}")

In [ ]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
test_features, test_labels = next(iter(test_loader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")

In [ ]:
#  Function to unnormalize and show an image
def imshow(img):
    img = (img *0.5) + 0.5  # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(10, 10))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# Get a batch of images and labels
dataiter = iter(train_dataloader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))

# Print labels
print(' '.join(f'{train_data.classes[labels[j]]}' for j in range(len(labels))))

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
class DenseLayer2(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super().__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        out = torch.cat([x, out], 1)
        return out

class DenseBlock1(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers):
        super().__init__()
        self.layer_num = n_layers
        self.deep_nn = nn.ModuleList([DenseLayer2(in_channels+i*growth_rate,growth_rate) for i in range(n_layers)])


    def forward(self, x):
        for i, l in enumerate(self.deep_nn):
            x = self.deep_nn[i](x)
        return x

class TransitionLayer1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2, stride=2)

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        out = self.pool(out)
        return out

In [ ]:
model = nn.Sequential(OrderedDict([
    ('Conv1',nn.Conv2d(3,32,kernel_size=7,padding=3,stride=2)),
    ('ReLU1',nn.ReLU()),
    ('MaxPool1',nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
    ('DenseBlock1',DenseBlock1(32,16,4)),
    ('TransitionLayer1',TransitionLayer1(96,48)),
    ('DenseBlock2',DenseBlock1(48,16,8)),
    ('TransitionLayer2',TransitionLayer1(176,88)),
    ('DenseBlock3',DenseBlock1(88,16,16)),
    ('TransitionLayer3',TransitionLayer1(344,172)),
    ('DenseBlock4',DenseBlock1(172,16,8)),
    ('Batchnorm1',nn.BatchNorm2d(300)),
    ('ReLU2',nn.ReLU()),
    ('Flatten',nn.Flatten()),
    ('Linear1',nn.Linear(300,128)),
    ('ReLU3',nn.ReLU()),
    ('Linear2',nn.Linear(128,6)),
]))

In [ ]:
summary(model,input_size=(1,3,48,48))

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, restore_best_weights=True, path='checkpoint.pt', verbose=False):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.path = path
        self.early_stop = False

    def __call__(self, val_loss, model):
        loss = val_loss

        if self.best_loss is None:
            self.best_loss = loss
            torch.save(model.state_dict(), self.path)
            # print("saved model1")
        elif loss - self.best_loss >=  self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = loss
            if self.restore_best_weights:
                torch.save(model.state_dict(), self.path)
            
            self.counter = 0

In [ ]:
# Create loss function
criterion = nn.CrossEntropyLoss()

# Create optimizer
optimizer = torch.optim.Adam(params=model.parameters(), # optimize newly created model's parameters
                            lr=0.001)